# Comment évoluent les relations entre les personnages de la saga Harry Potter?

###### Adrien Vallette et Sophie Vaillant

La saga Harry Potter est composée de 7 ouvrages mettant en scène une multitude de personnages. Au cours de ce projet, nous analyserons les interactions entre ces derniers, principalement en prenant pour référentiel Harry Potter, pour constater leur évolution quantitative et qualitative.

###### Packages et modules que nous utiliserons au cours de ce projet 

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display
import base64
import string
import re
from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from pywaffle import Waffle

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.collocations import *
try:
    stopwords = set(stopwords.words('english'))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stopwords = set(stopwords.words('english'))
import csv
import io
import spacy
import numpy as np

### Nettoyage des données 

Nous avons téléchargé les textes en anglais sur le lien suivant: https://github.com/formcept/whiteboard/tree/master/nbviewer/notebooks/data/harrypotter

##### Changement du format des textes

In [ ]:
#Tout d'abord, on crée un dictionnaire des différents tomes

Books = {'The Philosophers Stone': '1',
        'The Chamber of Secrets': '2',
        'The Prisoner of Azkaban': '3',
        'The Goblet of Fire':'4',
        'The Order of the Phoenix': '5',
        'The Half Blood Prince': '6',
        'The Deathly Hallows': '7'}

In [ ]:
#Création du CSV (pour Adrien)

def creation_csv(nom_livre,numero_livre):
    chemin = r"C:\Users\adxva\OneDrive\Bureau\ENSAE 2A - S1\Harry-Python\Data\Books txt\Book " + numero_livre + " - "  + nom_livre + ".txt"
    sortie = r"C:\Users\adxva\OneDrive\Bureau\ENSAE 2A - S1\Harry-Python\Data\Books CSV\Book " + numero_livre + ".csv"
    with io.open(chemin,"r",encoding="utf-8") as infile, open(sortie, 'w',encoding = 'utf-8-sig') as outfile:
        stripped = (line.strip() for line in infile)
        lines = (line.split(",") for line in stripped if line)
        writer = csv.writer(outfile)
        writer.writerows(lines)

In [ ]:
#On fait une boucle exécutant la fonction sur chaque élément du dictionnaire (pour Adrien)

Books_csv = {}
for title, i in Books.items():
    creation_csv(title, i)
    Books_csv['book_' + i] = pd.read_csv(r"C:\Users\adxva\OneDrive\Bureau\ENSAE 2A - S1\Harry-Python\Data\Books CSV\Book " + i + ".csv",encoding = 'utf-8-sig', sep='delimiter', header=None)

In [ ]:
#Création du CSV (pour Sophie)

def creation_csv(nom_livre,numero_livre):
    chemin = r"C:\Users\Sophie\Harry-Python\Data\Book " + numero_livre + " - " + nom_livre + ".txt"
    sortie = r"C:\Users\Sophie\Harry-Python\Data\book" + numero_livre + ".csv"
    with io.open(chemin,"r",encoding="utf-8") as infile, open(sortie, 'w',encoding = 'utf-8-sig') as outfile:
        stripped = (line.strip() for line in infile)
        lines = (line.split(",") for line in stripped if line)
        writer = csv.writer(outfile)
        writer.writerows(lines)

In [ ]:
#On fait une boucle exécutant la fonction sur chaque élément du dictionnaire (pour Sophie)

Books_csv = {}
for title, i in Books.items():
    creation_csv(title, i)
    Books_csv['book' + i] = pd.read_csv(r"C:\Users\Sophie\Harry-Python\Data\book" + i + ".csv" ,encoding = 'utf-8-sig', sep='delimiter', header=None)

In [ ]:
#Regardons ce que le CSV donne

Books_csv['book4'].head(100)

In [ ]:
#On crée un tableau avec tous les livres

df_books = pd.DataFrame(Books_csv.items(), columns = ['Books', 'Text'])
df_books

##### Nettoyage des textes

On souhaite supprimer les lignes en fin de pages qui apparaissent systématiquement, du type "Page 7 Harry Potter and the Philosophers Stone J.K. Rowling".
Pour cela on crée un pattern.

Nous avons pu constater en parcourant les textes qu'ils comportaient des fautes d'orthographe (lorsque l'on comparait les titres de chapitres observés aux vrais titres par exemple), et parfois des O à la place des 0 par exemple, ce qui explique par la suite des patterns parfois alambiqués.

In [ ]:
#On crée les patterns pour éliminer les lignes de fin de page qui reviennent dans chaque livre

pattern = ["Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Philosophers Stone[\s]?-[\s]?J.K. Rowling",
           "Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Chamber of Secrets[\s]?-[\s]?J.K. Rowling",
          "Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Prisoner of Azkaban[\s]?-[\s]?J.K. Rowling",
          "Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Goblet of Fire[\s]?-[\s]?J.K. Rowling",
          "Page[\s]?\|[\s]?[l0-9]?[lOU0-9]?[lOU0-9]?[lOU0-9][\s]?Harry Potter and the Order of the Phoenix[\s]?-[\s]?J.K. Rowling",
          "Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Half Blood Prince[\s]?-[\s]?J.K. Rowling",
          "Page[\s]?\|[\s]?[0-9]?[0-9]?[0-9]?[0-9][\s]?Harry Potter and the Deathly Hallows[\s]?-[\s]?J.K. Rowling"]

In [ ]:
#On crée une fonction permettant de supprimer ces lignes lorsque le pattern est rencontré au cours du texte

def clean_page(text, pattern):
    for ele in pattern:
        liste_regex = re.findall(ele, text, re.IGNORECASE)
        for expression in liste_regex:
            text = text.replace(expression, '')
    return text

In [ ]:
#On crée le même dictionnaire en fusionnant chaque texte du CSV pour pouvoir appliquer la fonction

all_text = Books_csv.copy()
for i in Books_csv:
    all_text[i] = ' '.join([text for text in Books_csv[i][0]])

In [ ]:
#On applique la fonction au dictionnaire des livres afin de supprimer les patterns que l'on rencontre

for i in all_text:
    all_text[i] = clean_page(all_text[i], pattern)

On réalise une tokenisation sur le texte pour enlever les mots qui ne nous intéressent pas.

In [ ]:
#nltk.download('punkt')

all_text_clean = all_text.copy()

for i in all_text:
    
    text_list = nltk.tokenize.word_tokenize(all_text[i])
    text_list = [x.lower() for x in text_list]
    text_clean = [w.lower() for w in text_list if w not in stopwords and w.isalpha()]
    text_clean = ' '.join(text.lower() for text in text_clean)
    all_text_clean[i] = text_clean

In [ ]:
#On regarde ce que ça donne sur un bout de texte

all_text_clean['book5'][:1000]

On obtient ainsi les données "all_text_clean" qui sont les textes nettoyés et regroupés en un dictionnaire.

### Statistiques descriptives

On veut regarder les mots qui reviennent le plus dans toute la saga: on réalise des wordclouds et des graphiques pour chaque tome.

In [ ]:
#Création des wordclouds

for i in all_text_clean:
    wordcloud = WordCloud(width=800, height=500,
                      random_state=21, max_font_size=110).generate(all_text_clean[i])
    plt.figure(figsize=(15, 12))
    plt.title(i)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off');

In [ ]:
#On crée une fonction qui détermine les mots qui reviennent le plus dans un texte ainsi que leur nombre d'apparition

def most_common_word(text_clean):
    text_clean_list = text_clean.split()
    text_counts = Counter(text_clean_list)
    text_common_words = [word[0] for word in text_counts.most_common(25)]
    text_common_counts = [word[1] for word in text_counts.most_common(25)]

    return text_common_words, text_common_counts

In [ ]:
#Création d'un dictionnaire avec les mots les plus utilisés par tome

hp_most_common = {}
for i in all_text_clean:
    hp_most_common[i] = most_common_word(all_text_clean[i])

In [ ]:
#On réalise les graphiques des mots les plus communs

for i in hp_most_common:
        plt.style.use('dark_background')
        plt.figure(figsize=(15, 12))
        words = hp_most_common[i][0]
        count = hp_most_common[i][1]
        sns.barplot(x = words, y = count)
        plt.title('Most Common Words used by J.K. Rowling in ' +i)

On va maintenant travailler plus spécifiquement sur les personnages.

In [ ]:
all_text_clean_list = all_text_clean.copy()
for i in all_text_clean:
    all_text_clean_list[i] = all_text_clean[i].split()


harry = ['harry', 'harry potter']
ron = ['ron', 'ron weasley']
hermione = ['hermione granger','hermione']
ginny = ['ginny weasley', 'ginny']
dumbledore = ['albus dumbledore', 'dumbledore']
sirius = ['sirius black', 'sirius']
snape = ['severus snape', 'snape']
luna = ['luna lovegood', 'luna']
hagrid = ['rubeus hagrid', 'hagrid']
dobby = ['dobby']
voldemort = ['tom riddle', 'voldemort', 'you-know-who', 'know-who', 'lord']